## Analyze whether SNWD varies more from year to year or from place to place.

In [5]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBSSBBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
502
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|             res_2|               res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+------------------+--------------------+------------------+-----------+------------+------+--------------------+------+
| -8864.63634612004|-2039.954886131679| 698.3849072461817|   1797.4|SBSSBBBS|  43.465|-113.5581|       SNWD|0.08489938379981538| 0.035341730561038|0.030868023647585202|0.748963242539

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [33]:
#extract longitude and latitude for each station
# feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
# Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
def MS(Mat):
    return np.nanmean(Mat**2)
for i in range(3):
    stri = str(i+1)
    Features='station, year, coeff_'+stri
    Query="SELECT %s FROM weather"%Features
    print(Query)
    pdf = sqlContext.sql(Query).toPandas()
    print pdf.head()
    year_station_table=pdf.pivot(index='year', columns='station', values='coeff_'+stri)
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station

    print 'total MS                   = ', MS(year_station_table)
    print 'MS removing mean-by-station= ', MS(tbl_minus_station)
    print 'MS removing mean-by-year   = ', MS(tbl_minus_year)
    print '==============='

SELECT station, year, coeff_1 FROM weather
       station    year      coeff_1
0  USC00102260  1993.0 -8864.636346
1  USC00104845  1997.0 -6921.010051
2  USC00104845  1980.0 -5862.926829
3  USC00104230  1972.0 -6048.058037
4  USC00103942  1952.0 -9637.720618
total MS                   =  7645869.4234
MS removing mean-by-station=  2337260.72853
MS removing mean-by-year   =  4600023.34546
SELECT station, year, coeff_2 FROM weather
       station    year      coeff_2
0  USC00102260  1993.0 -2039.954886
1  USC00104845  1997.0  -665.958256
2  USC00104845  1980.0  1176.284964
3  USC00104230  1972.0 -1602.792073
4  USC00103942  1952.0   145.355169
total MS                   =  720290.294619
MS removing mean-by-station=  686028.439632
MS removing mean-by-year   =  292349.048244
SELECT station, year, coeff_3 FROM weather
       station    year      coeff_3
0  USC00102260  1993.0   698.384907
1  USC00104845  1997.0   270.230987
2  USC00104845  1980.0   302.271603
3  USC00104230  1972.0 -1557.231

In [13]:

# year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
# year_station_table.head(10)

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [30]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))



mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

 total RMS                   =  848.69917793
RMS removing mean-by-station=  828.268337939
RMS removing mean-by-year   =  540.693118362


In [9]:

T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 848.69917793
0 after removing mean by year    = 540.693118362
0 after removing mean by stations= 526.181161298
1 after removing mean by year    = 524.635943663
1 after removing mean by stations= 524.321748828
2 after removing mean by year    = 524.224364916
2 after removing mean by stations= 524.186093351
3 after removing mean by year    = 524.169321724
3 after removing mean by stations= 524.161576845
4 after removing mean by year    = 524.157893966
4 after removing mean by stations= 524.156109745
